# 5 Thomson-Reuters Tick History intraday data


In [1]:
import polars as pl
import numpy as np

import glob

allfiles=glob.glob("data/raw/TRTH/equities/US/*trade*")

### 1. Data exploration

In [2]:
print(allfiles[0])

DF=pl.read_csv(allfiles[0])
DF

IndexError: list index out of range

xltime is a timestamp in a peculiar format; trade-price and trade-volume are straightforward; trade-stringflag deserves further study; trade_rawflag is clearly quite complex and will be left out of the discussion.

There are quite many lines for a single day of trade: SPY is one of the most traded ETFs.

Let us see now what kinds of trade_stringflag there are:

In [10]:
DF["trade-stringflag"].value_counts()

trade-stringflag,count
str,u32
"""blocktrade""",467
"""blocktrade|auction""",13
"""blocktrade|marketclosed|volume…",41
"""marketclosed|volumeupdate""",9650
"""theoricalprice""",1
"""uncategorized""",200878
"""auction""",341


Let us only keep the uncategorized (read: non-special) lines

In [11]:
DF=DF.filter(pl.col("trade-stringflag")=="uncategorized")

### 2. Timestamp



In [12]:
import polars as pl

# Define a reference date (Excel-style)
excel_base_date = pl.datetime(1899, 12, 30)  # Excel starts counting from 1900-01-01, but Polars needs 1899-12-30

# Convert the float days to a proper date
DF = DF.with_columns(
    (pl.col("xltime") * pl.duration(days=1) + excel_base_date).alias("index")
)
DF = DF.with_columns(pl.col("index").dt.convert_time_zone("America/New_York"))


In [13]:

DF.drop(["xltime","trade-rawflag","trade-stringflag"])


trade-price,trade-volume,index
f64,i64,"datetime[μs, America/New_York]"
114.89,100,2010-05-18 09:30:01.221 EDT
114.89,100,2010-05-18 09:30:01.221 EDT
114.89,100,2010-05-18 09:30:01.229 EDT
114.89,200,2010-05-18 09:30:01.229 EDT
114.89,200,2010-05-18 09:30:01.229 EDT
…,…,…
112.42,400,2010-05-18 16:00:00.610 EDT
112.42,300,2010-05-18 16:00:00.610 EDT
112.42,700,2010-05-18 16:00:00.610 EDT


### 3. Function

In [14]:
def load_TRTH_trade(filename,
            tz_exchange="America/New_York",
            only_non_special_trades=True,
            only_regular_trading_hours=True,
            merge_sub_trades=True):
    try:
        if filename.endswith("csv") or filename.endswith("csv.gz"):
            DF=pl.read_csv(filename)
        elif filename.endswith("parquet"):    
            DF=pl.read_parquet(filename)
        else:
            print("cannot load file "+filename+" : unknown format")
            return None
    except:
        print(filename+" cannot be loaded")
        return None

    excel_base_date = pl.datetime(1899, 12, 30)  # Excel starts counting from 1900-01-01, but Polars needs 1899-12-30
    DF = DF.with_columns(
        (pl.col("xltime") * pl.duration(days=1) + excel_base_date).alias("index")
    )
    DF = DF.with_columns(pl.col("index").dt.convert_time_zone(tz_exchange))
    DF.drop(["xltime","trade-rawflag","trade-stringflag"])

    if only_non_special_trades:
        DF=DF.filter(pl.col("trade-stringflag")=="uncategorized")

    if merge_sub_trades:   # average volume-weighted trade price here
        DF=DF.group_by('index',maintain_order=True).agg([(pl.col('trade-price')*pl.col('trade-volume')).sum()/(pl.col('trade-volume').sum()).alias('trade-price'),pl.sum('trade-volume')])        
    
    return DF



In [1]:
def load_TRTH_bbo(filename,
            tz_exchange="America/New_York",
            only_regular_trading_hours=True,
            hhmmss_open="09:30:00",
            hhmmss_close="16:00:00",
            merge_same_index=True):
    try:
        if filename.endswith("csv") or filename.endswith("csv.gz"):
            DF=pl.read_csv(filename)
        elif filename.endswith("parquet"):    
            DF=pl.read_parquet(filename)
        else:
            print("cannot load file "+filename+" : unknown format")
            return None
    except:
        print(filename+" cannot be loaded")
        return None

    excel_base_date = pl.datetime(1899, 12, 30)  # Excel starts counting from 1900-01-01, but Polars needs 1899-12-30
    DF = DF.with_columns(
        (pl.col("xltime") * pl.duration(days=1) + excel_base_date).alias("index")
    )
    DF = DF.with_columns(pl.col("index").dt.convert_time_zone(tz_exchange))
    DF.drop("xltime")

    # apply common sense filter
    DF = DF.filter(pl.col("ask-price")>0).filter(pl.col("bid-price")>0).filter(pl.col("ask-price")>pl.col("bid-price"))

    if merge_same_index:
        DF = DF.group_by('index',maintain_order=True).last()   # last quote of the same timestamp
    
    if only_regular_trading_hours:
        hh_open,mm_open,ss_open = [float(x) for x in hhmmss_open.split(":")]
        hh_close,mm_close,ss_close = [float(x) for x in hhmmss_close.split(":")]

        seconds_open=hh_open*3600+mm_open*60+ss_open
        seconds_close=hh_close*3600+mm_close*60+ss_close

        DF = DF.filter(pl.col('index').dt.hour().cast(float)*3600+pl.col('index').dt.minute().cast(float)*60+pl.col('index').dt.second()>=seconds_open,
                       pl.col('index').dt.hour().cast(float)*3600+pl.col('index').dt.minute().cast(float)*60+pl.col('index').dt.second()<=seconds_close)
    
    return DF



In [33]:
load_TRTH_trade("/data/finance/TRTH/raw/equities/US/trade/SPY.P/2009-01-07-SPY.P-trade.parquet")

by,trade-price,trade-volume
"datetime[μs, America/New_York]",f64,i32
2009-01-07 09:30:02.045 EST,92.0,500
2009-01-07 09:30:02.054999 EST,92.0,2427
2009-01-07 09:30:02.064999 EST,92.0,5378
2009-01-07 09:30:02.105 EST,92.0,3541
2009-01-07 09:30:02.108999 EST,92.0,11422
…,…,…
2009-01-07 16:00:01.065 EST,90.641931,12945
2009-01-07 16:00:01.087 EST,90.62,6078204
2009-01-07 16:00:01.098 EST,90.62,147663
